In [ ]:
import yfinance as yf
import pandas as pd
import ta

# Baixar os últimos 30 dias de dados da Apple
ticker = "AAPL"
dados = yf.download(ticker, period="30d", interval="1d")

# Garantir que "Close" seja uma série 1D
dados["Close"] = dados["Close"].astype(float).squeeze()

# Calcular a Média Móvel Simples (SMA) de 10 e 20 dias
dados["SMA_10"] = dados["Close"].rolling(window=10).mean()
dados["SMA_20"] = dados["Close"].rolling(window=20).mean()

# Calcular o RSI (Relative Strength Index) de 14 dias
rsi_indicator = ta.momentum.RSIIndicator(close=dados["Close"].squeeze(), window=14)
dados["RSI"] = rsi_indicator.rsi()

# Calcular as Bandas de Bollinger (20 dias)
bollinger = ta.volatility.BollingerBands(close=dados["Close"].squeeze(), window=20)
dados["Bollinger_High"] = bollinger.bollinger_hband()
dados["Bollinger_Low"] = bollinger.bollinger_lband()


def gerar_recomendacao(df):
    if len(df) < 20:
        return "Dados insuficientes para análise."

    ult_linha = df.iloc[-1]  # Última linha corretamente indexada
    recomendacoes = []

    # 🔹 Verificando se "RSI" existe e não é NaN antes da análise
    if "RSI" in ult_linha and not pd.isna(ult_linha["RSI"]):
        rsi_valor = ult_linha["RSI"].item()  # Converte para um número escalar
        if rsi_valor < 30:
            recomendacoes.append("RSI indica COMPRA (ativo sobrevendido).")
        elif rsi_valor > 70:
            recomendacoes.append("RSI indica VENDA (ativo sobrecomprado).")

    # 🔹 Verificando cruzamento de médias móveis
    if len(df) > 1:
        prev_linha = df.iloc[-2]
        if "SMA_10" in ult_linha and "SMA_20" in ult_linha:
            if prev_linha["SMA_10"] < prev_linha["SMA_20"] and ult_linha["SMA_10"] > ult_linha["SMA_20"]:
                recomendacoes.append("Cruzamento de Médias Móveis indica COMPRA.")
            elif prev_linha["SMA_10"] > prev_linha["SMA_20"] and ult_linha["SMA_10"] < ult_linha["SMA_20"]:
                recomendacoes.append("Cruzamento de Médias Móveis indica VENDA.")

    # 🔹 Analisando Bandas de Bollinger
    if "Close" in ult_linha and "Bollinger_Low" in ult_linha and "Bollinger_High" in ult_linha:
        close_valor = ult_linha["Close"].item()
        bollinger_low = ult_linha["Bollinger_Low"].item()
        bollinger_high = ult_linha["Bollinger_High"].item()

        if close_valor < bollinger_low:
            recomendacoes.append("Banda de Bollinger indica COMPRA (preço abaixo da banda inferior).")
        elif close_valor > bollinger_high:
            recomendacoes.append("Banda de Bollinger indica VENDA (preço acima da banda superior).")

    return "\n".join(recomendacoes) if recomendacoes else "Nenhuma recomendação clara no momento."

# Gerar e exibir recomendação
recomendacao = gerar_recomendacao(dados)
print("\nRecomendação baseada nos indicadores:")
print(recomendacao)